In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# test

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms

In [ ]:
import torch
import numpy as np
import skimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(1)  # reproducible
torch.set_default_tensor_type(torch.DoubleTensor)
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])
'''NPY数据格式'''
# class MyDataset(Dataset):
#     def __init__(self, data,label):
#         self.data = np.load(data) #加载npy数据
#         # self.data=self.data
#         self.label = np.load(label)
#         self.transforms = transform #转为tensor形式
#     def __getitem__(self, index):
#         hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
#         # print(hdct.shape)
#         # hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         # ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         # hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         # ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
#         hdct = hdct.astype("float")
#         hdct= self.transforms(hdct)  #转为tensor形式
#         # ldct= self.transforms(ldct)  #转为tensor形式？??
#         return hdct, self.label[index] #返回数据还有标签
#     def __len__(self):
#         return self.data.shape[0] #返回数据的总个数
 
# def main():
#     dataset=MyDataset('train1.npy',"label.npy")
#     data= DataLoader(dataset, batch_size=8, shuffle=True, pin_memory=True)
# if __name__ == '__main__':
#     main()

'NPY数据格式'

In [ ]:

class MyDataset(Dataset):
    def __init__(self, data,label):
        self.data = data #加载npy数据
        self.label = label
        self.transforms = transform #转为tensor形式
    def __getitem__(self, index):
        hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
        hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
        hdct= self.transforms(hdct)  #转为tensor形式
#         ldct= self.transforms(ldct)  #转为tensor形式
        return hdct, self.label[index] #返回数据还有标签
    def __len__(self):
        return self.data.shape[0] #返回数据的总个数



In [ ]:
# dataset=MyDataset('/content/drive/MyDrive/100/test3/x_train.npy',"/content/drive/MyDrive/100/test3/y_train.npy")
# dataset_test=MyDataset('/content/drive/MyDrive/100/test3/x_test.npy',"/content/drive/MyDrive/100/test3/y_test.npy")
# train_loader= DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
# test_loader= DataLoader(dataset_test, batch_size=128, shuffle=False, pin_memory=True)

In [ ]:

__all__ = ['ResNet50', 'ResNet101','ResNet152']

def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=8, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(8192,200)
        self.fc2 = nn.Linear(200,num_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.fc2(x)
        
        return self.logsoftmax(x)

def ResNet50():
    return ResNet([3, 4, 6, 3])

def ResNet101():
    return ResNet([3, 4, 23, 3])

def ResNet152():
    return ResNet([3, 8, 36, 3])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model = ResNet50().to(device)

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.008,momentum=0.5)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold as kFold

In [ ]:
x_train=np.load("/content/drive/MyDrive/summer_start/data_generate/train_3channel.npy")
y_train=np.load("/content/drive/MyDrive/summer_start/data_generate/label.npy")

In [ ]:
# da1=np.load('/content/drive/MyDrive/100/test3/x_test.npy')
x_train=x_train.reshape(13887,236,236,-1)

In [ ]:
x_train.shape

(13887, 236, 236, 3)

In [ ]:
kfold =kFold(n_splits=5,shuffle=True,random_state=1)

In [ ]:
import time

In [ ]:
# labels=[0,1,2,3,4,5,6,7]
# list_score=[]
# for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
#     print("train_index",train_index)
#     print("test_index",test_index)
#     ### Dividing data into folds
#     x_train_fold = x_train[train_index]
#     x_test_fold = x_train[test_index]
#     y_train_fold = y_train[train_index]
#     y_test_fold = y_train[test_index]
#     train = MyDataset(x_train_fold, y_train_fold)
#     test = MyDataset(x_test_fold, y_test_fold)
#     train_loader = DataLoader(train, batch_size = 8, shuffle = True)
#     test_loader = DataLoader(test, batch_size = 8, shuffle = True)
#     print("fold:",fold)
#     max_score=[]
#     # model = Net().to(device)
#     best_acc=0
#     model = ResNet50().to(device)
#     loss_fn = nn.NLLLoss()
#     optimizer = torch.optim.SGD(model.parameters(),lr=0.005,momentum=0.5)
#     for i in range(30):
#       list1=[]
#       for t, (data, target) in enumerate(train_loader):
#         # print("t",t)
#         data,target = Variable(data.to(device)),Variable(target.to(device))
#         # print(data.shape)
#         pred = model(data.double().to(device))
#         loss = loss_fn(pred,target)
        
#         list1.append(loss.item())
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # print("loss",loss.item())
#       print("netloss",i,np.mean(list1))
#       with torch.no_grad():
#         list_loss=[]
#         correct = 0
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data.double().to(device))
#             # get the index of the max log-probability
#             pred = output.data.max(1, keepdim=True)[1]
#             pred_loss=np.exp(output.data.cpu())
#             logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
#             list_loss.append(logsloss)
#             correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#         print('accuracy:{:.3f}%'.format(
#             100. * correct / len(test_loader.dataset)))
#         val_acc=correct / len(test_loader.dataset)
#         print("log_loss",sum(list_loss)/len(list_loss))
#         print("*"*20)
#         if val_acc > best_acc:
#             best_acc = val_acc
#             es = 0
#         else:
#             es += 1
#             print("Counter {} of 5".format(es))
#             if es > 4:
#                 print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
#                 break
#     list_score.append(best_acc)
#     print("*"*40)
#     del train,x_train_fold,train_loader,test_loader,test,y_test_fold,x_test_fold,y_train_fold
#     import gc
#     gc.collect()
#     time.sleep(20)
# from numpy import mean
# print(mean(list_score))


train_index [    0     1     2 ... 13884 13885 13886]
test_index [    4     5     6 ... 13861 13872 13877]
fold: 0
netloss 0 1.4712663527459429
accuracy:50.972%
log_loss 1.4013622103169192
********************
netloss 1 1.0924889784705276
accuracy:65.587%
log_loss 1.074192525346613
********************
netloss 2 0.8794961562271655
accuracy:69.582%
log_loss 0.9445064034740145
********************
netloss 3 0.7700175565007278
accuracy:72.714%
log_loss 0.9106128648186768
********************
netloss 4 0.6968674937831847
accuracy:79.770%
log_loss 0.6403219766237334
********************
netloss 5 0.645062826362993
accuracy:80.994%
log_loss 0.6114341181035724
********************
netloss 6 0.6051424559116392
accuracy:78.330%
log_loss 0.6645491096155313
********************
Counter 1 of 5
netloss 7 0.5718186300762017
accuracy:83.405%
log_loss 0.5788679055667972
********************
netloss 8 0.5391746339577858
accuracy:83.189%
log_loss 0.5300530272363388
********************
Counter 1 of 5
ne

In [ ]:
mean(list_score)

In [ ]:
# labels=[0,1,2,3,4,5,6,7]
# list_score=[]
# for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
#   if fold ==0:
#     pass
#   else:
#     print("train_index",train_index)
#     print("test_index",test_index)
#     ### Dividing data into folds
#     x_train_fold = x_train[train_index]
#     x_test_fold = x_train[test_index]
#     y_train_fold = y_train[train_index]
#     y_test_fold = y_train[test_index]
#     train = MyDataset(x_train_fold, y_train_fold)
#     test = MyDataset(x_test_fold, y_test_fold)
#     train_loader = DataLoader(train, batch_size = 8, shuffle = True)
#     test_loader = DataLoader(test, batch_size = 8, shuffle = True)
#     print("fold:",fold)
#     max_score=[]
#     # model = Net().to(device)
#     best_acc=0
#     model = ResNet50().to(device)
#     loss_fn = nn.NLLLoss()
#     optimizer = torch.optim.SGD(model.parameters(),lr=0.005,momentum=0.5)
#     for i in range(30):
#       list1=[]
#       for t, (data, target) in enumerate(train_loader):
#         # print("t",t)
#         data,target = Variable(data.to(device)),Variable(target.to(device))
#         # print(data.shape)
#         pred = model(data.double().to(device))
#         loss = loss_fn(pred,target)
        
#         list1.append(loss.item())
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # print("loss",loss.item())
#       print("netloss",i,np.mean(list1))
#       with torch.no_grad():
#         list_loss=[]
#         correct = 0
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data.double().to(device))
#             # get the index of the max log-probability
#             pred = output.data.max(1, keepdim=True)[1]
#             pred_loss=np.exp(output.data.cpu())
#             logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
#             list_loss.append(logsloss)
#             correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#         print('accuracy:{:.3f}%'.format(
#             100. * correct / len(test_loader.dataset)))
#         val_acc=correct / len(test_loader.dataset)
#         print("log_loss",sum(list_loss)/len(list_loss))
#         print("*"*20)
#         if val_acc > best_acc:
#             best_acc = val_acc
#             es = 0
#         else:
#             es += 1
#             print("Counter {} of 5".format(es))
#             if es > 4:
#                 print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
#                 break
#     list_score.append(best_acc)
#     print("*"*40)
#     del train,x_train_fold,train_loader,test_loader,test,y_test_fold,x_test_fold,y_train_fold
#     import gc
#     gc.collect()
#     time.sleep(20)
# from numpy import mean
# print(mean(list_score))


train_index [    0     1     2 ... 13883 13884 13885]
test_index [   16    21    24 ... 13866 13867 13886]
fold: 1
netloss 0 1.4784420477979607
accuracy:55.400%
log_loss 1.2171844087043378
********************
netloss 1 1.0848883827824456
accuracy:70.842%
log_loss 0.8524711350011506
********************
netloss 2 0.8772512802359188
accuracy:72.822%
log_loss 0.8526367537384774
********************
netloss 3 0.7842084842655183
accuracy:75.594%
log_loss 0.718781070371376
********************
netloss 4 0.7162441988572403
accuracy:77.394%
log_loss 0.667001896657387
********************
netloss 5 0.6552038287296282
accuracy:79.770%
log_loss 0.6517169193468064
********************
netloss 6 0.6143899162015036
accuracy:82.541%
log_loss 0.560457954582815
********************
netloss 7 0.5714481710327588
accuracy:80.922%
log_loss 0.5651968160807422
********************
Counter 1 of 5
netloss 8 0.551115827334188
accuracy:81.246%
log_loss 0.5484642527836391
********************
Counter 2 of 5
netl

In [ ]:
labels=[0,1,2,3,4,5,6,7]
list_score=[]
for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
  if fold in[0,1,2]:
    pass
  else:
    print("train_index",train_index)
    print("test_index",test_index)
    ### Dividing data into folds
    x_train_fold = x_train[train_index]
    x_test_fold = x_train[test_index]
    y_train_fold = y_train[train_index]
    y_test_fold = y_train[test_index]
    train = MyDataset(x_train_fold, y_train_fold)
    test = MyDataset(x_test_fold, y_test_fold)
    train_loader = DataLoader(train, batch_size = 8, shuffle = True)
    test_loader = DataLoader(test, batch_size = 8, shuffle = True)
    print("fold:",fold)
    max_score=[]
    # model = Net().to(device)
    best_acc=0
    model = ResNet50().to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=0.005,momentum=0.5)
    for i in range(30):
      list1=[]
      for t, (data, target) in enumerate(train_loader):
        # print("t",t)
        data,target = Variable(data.to(device)),Variable(target.to(device))
        # print(data.shape)
        pred = model(data.double().to(device))
        loss = loss_fn(pred,target)
        
        list1.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("loss",loss.item())
      print("netloss",i,np.mean(list1))
      with torch.no_grad():
        list_loss=[]
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double().to(device))
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            pred_loss=np.exp(output.data.cpu())
            logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
            list_loss.append(logsloss)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('accuracy:{:.3f}%'.format(
            100. * correct / len(test_loader.dataset)))
        val_acc=correct / len(test_loader.dataset)
        print("log_loss",sum(list_loss)/len(list_loss))
        print("*"*20)
        if val_acc > best_acc:
            best_acc = val_acc
            es = 0
        else:
            es += 1
            print("Counter {} of 5".format(es))
            if es > 4:
                print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
                break
    list_score.append(best_acc)
    print("*"*40)
    del train,x_train_fold,train_loader,test_loader,test,y_test_fold,x_test_fold,y_train_fold
    import gc
    gc.collect()
    time.sleep(20)
from numpy import mean
print(mean(list_score))


train_index [    0     1     2 ... 13883 13884 13886]
test_index [    8     9    13 ... 13876 13882 13885]
fold: 3
netloss 0 1.4810784750848907
accuracy:54.519%
log_loss 1.3081312726140788
********************
netloss 1 1.128782950623477
accuracy:68.023%
log_loss 0.9337413279062241
********************
netloss 2 0.8921824476366864
accuracy:71.264%
log_loss 0.8953130885018485
********************
netloss 3 0.763650426631659
accuracy:76.125%
log_loss 0.7634081970117871
********************
netloss 4 0.6922205174861856
accuracy:78.322%
log_loss 0.72829968362921
********************
netloss 5 0.6340897150151344
accuracy:79.078%
log_loss 0.6679522739483912
********************
netloss 6 0.5946184273867529
accuracy:79.690%
log_loss 0.6153683009006228
********************
netloss 7 0.5533235156933477
accuracy:77.170%
log_loss 0.6893562519319217
********************
Counter 1 of 5
netloss 8 0.5203606670560383
accuracy:81.275%
log_loss 0.5947676213544063
********************
netloss 9 0.4964030

In [1]:
(0.8655023406553835*2+0.8718+0.8254+0.8614)/5

0.8579209362621534